# Daily COVID monitoring data for Santa Barbara county

In [1]:
# Preamble
import os
import IPython
import requests
from bs4 import BeautifulSoup
import pandas as pd
import datetime
import gspread
from oauth2client.service_account import ServiceAccountCredentials

In [2]:
# Change working directory (so that Windows task scheduler does not complain)
os.chdir("C:/Users/Alice/Box Sync/PhD/Software/Python/COVID/COVID-SB")

In [3]:
# Request content from URL
html_string = requests.get('https://www.cdph.ca.gov/Programs/CID/DCDC/Pages/COVID-19/COVID19CountyDataTable.aspx').content

In [4]:
# Parse HTML
soup = BeautifulSoup(html_string, 'html.parser')

In [5]:
# Extract data for Santa Barbara
table = soup.find_all("tr", class_="ms-rteTableEvenRow-3")
SB = table[21]

In [6]:
# Avg # tests per day (per 100,000 population)
daily_tests = SB.find_all("td", class_="ms-rteTableOddCol-3")[0].text

# Case rate per 100,000
case_rate = SB.find_all("td", class_="ms-rteTableEvenCol-3")[1].text

# Testing positivity (%)
test_positivity = SB.find_all("td", class_="ms-rteTableOddCol-3")[1].text

# % Change in 3-day avg COVID+ hospitalized patients
change_hospit = SB.find_all("td", class_="ms-rteTableEvenCol-3")[2].text

# % ICU beds currently available
ICU_beds = SB.find_all("td", class_="ms-rteTableOddCol-3")[2].text

# % Ventilators currently available
ventilators = SB.find_all("td", class_="ms-rteTableEvenCol-3")[3].text

In [7]:
# table = pd.DataFrame({"Date": [date.today().strftime("%y/%m/%d")],
#                       "Daily tests (per 100,000)": [daily_tests], 
#                       "Case rate (per 100,000)": [case_rate],
#                       "Testing positivity (%)": [test_positivity],
#                       "Change in hospitalizations (%)": [change_hospit], 
#                       "ICU beds available (%)": [ICU_beds],
#                       "Ventilators available (%)": [ventilators]})
# table.set_index("Date")

In [8]:
# Set up Google Sheet API
scope = ["https://spreadsheets.google.com/feeds",'https://www.googleapis.com/auth/spreadsheets',"https://www.googleapis.com/auth/drive.file","https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name("credentials.json", scope)
client = gspread.authorize(creds)

In [9]:
# Append daily data as a new row in Google Sheet
sheet = client.open("COVID Daily Metrics").sheet1
insertRow = [datetime.date.today().strftime("%y/%m/%d"),
             daily_tests,
             case_rate,
             test_positivity,
             change_hospit,
             ICU_beds,
             ventilators]
sheet.insert_row(insertRow, 2)

{'spreadsheetId': '13GG0FPUFg_5xCouN0ZF3woaPXwjuxL2Dw3uYIfwUW-Q',
 'updatedRange': 'Sheet1!A2:G2',
 'updatedRows': 1,
 'updatedColumns': 7,
 'updatedCells': 7}

In [10]:
# Extract data from Google Sheet as pandas dataframe
sheet = client.open("COVID Daily Metrics").sheet1
data = sheet.get_all_records()
result = pd.DataFrame(data).set_index("Date")

In [11]:
result

,"Daily tests (per 100,000)¹","Case rate (per 100,000)²",Testing positivity (%)³,Change in hospitalizations (%)⁴,ICU beds available (%),Ventilators (available (%)
Date,,,,,,
20/08/21,214.5,148.8,✔,✔,✔,✔
20/08/20,214.5,148.8,✔,✔,✔,✔


Notes:
1. 7 day average with a 7-day lag.
2. 14 day average.
3. 7 day average with a 7-day lag.
4. Change in 3 day average.

Source: https://www.cdph.ca.gov/Programs/CID/DCDC/Pages/COVID-19/COVID19CountyDataTable.aspx.

See the code to scrape the data: https://github.com/walice/COVID-SB/.

In [12]:
%%javascript
IPython.notebook.save_notebook()

<IPython.core.display.Javascript object>

In [13]:
# Convert Jupyter Notebook to HTML
! jupyter nbconvert "index.ipynb" --to html --TagRemovePreprocessor.enabled=True --TagRemovePreprocessor.remove_cell_tags="['remove_cell']" --TagRemovePreprocessor.remove_input_tags="['remove_input']" --no-prompt

[NbConvertApp] Converting notebook index.ipynb to html
[NbConvertApp] Writing 274171 bytes to index.html


In [14]:
# Create daily commit message
now = datetime.datetime.now()
commit_message = "Last scraped on " + str(now)

In [ ]:
# Deploy to GitHub
! git add .
! echo $commit_message > commit_message.txt
! git commit -F commit_message.txt
! git push